<a href="https://colab.research.google.com/github/kaviya2478/100_days_ML/blob/main/Bank_Statement_Analyzer_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas camelot-py tabula-py pdfplumber PyPDF2 pytesseract


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of pdfplumber to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.1 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of pdfplumber to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtrackin

##Extracting Data From PDF

In [14]:
import camelot

pdf_path = "Synthetic Data 1.pdf"
tables = camelot.read_pdf(pdf_path, pages="all")

df = tables[0].df
df.columns = df.iloc[0]
df = df[1:]
df.reset_index(drop=True, inplace=True)

print(df.head())
df.to_csv("extracted_transactions.csv", index=False)



0 S. No\nCheque/Ref No\nValue Date\nDate\nTransaction \nWithdrawal (Dr)\nDeposit (Cr)\nRemarks  \
0                                                  1                                             
1                                                  2                                             
2                                                  3                                             
3                                                  4                                             
4                                                  5                                             

0                                                          \
0  436251  01-Sep-2024 \n09:00:23  01-Sep-2024 \n09:00:23   
1  936412  01-Sep-2024 \n09:05:02  01-Sep-2024 \n09:05:02   
2  351286  01-Sep-2024 \n10:12:45  01-Sep-2024 \n10:12:45   
3  125874  01-Sep-2024 \n11:30:11  01-Sep-2024 \n11:30:11   
4  412398  02-Sep-2024 \n08:45:37  02-Sep-2024 \n08:45:37   

0                                           

In [5]:
print(df.columns)  # Print existing column names
print(df.head())  # Preview first few rows
print(df.shape)  # Show number of rows & columns


Index(['S. No\nCheque/Ref No\nValue Date\nDate\nTransaction \nWithdrawal (Dr)\nDeposit (Cr)\nRemarks',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6'],
      dtype='object')
   S. No\nCheque/Ref No\nValue Date\nDate\nTransaction \nWithdrawal (Dr)\nDeposit (Cr)\nRemarks  \
0                                                  1                                              
1                                                  2                                              
2                                                  3                                              
3                                                  4                                              
4                                                  5                                              

   Unnamed: 1              Unnamed: 2              Unnamed: 3  \
0      436251  01-Sep-2024 \n09:00:23  01-Sep-2024 \n09:00:23   
1      936412  01-Sep-2024 \n09:05:02  01-Sep-2024 

##Cleaning the data


In [6]:
expected_columns = ["Date", "Time", "Description", "Transaction Type", "Amount", "Balance"]

if len(df.columns) > len(expected_columns):
    df = df.iloc[:, :len(expected_columns)]

df.columns = expected_columns


In [15]:
import pandas as pd

df = pd.read_csv("extracted_transactions.csv")


df = df.rename(columns={
    "Unnamed: 1": "Cheque/Ref No",
    "Unnamed: 2": "Value Date",
    "Unnamed: 3": "Date",
    "Unnamed: 4": "Transaction Details",
    "Unnamed: 5": "Withdrawal (Dr)",
    "Unnamed: 6": "Deposit (Cr)"
})

df = df[["Date", "Transaction Details", "Withdrawal (Dr)", "Deposit (Cr)"]]

df["Date"] = pd.to_datetime(df["Date"].str.strip(), errors="coerce").dt.strftime("%Y-%m-%d")

df.fillna({"Withdrawal (Dr)": 0, "Deposit (Cr)": 0}, inplace=True)


df["Withdrawal (Dr)"] = df["Withdrawal (Dr)"].replace(r"[^\d.]", "", regex=True).astype(float)
df["Deposit (Cr)"] = df["Deposit (Cr)"].replace(r"[^\d.]", "", regex=True).astype(float)


df.to_csv("cleaned_transactions.csv", index=False)
df.to_json("cleaned_transactions.json", orient="records", indent=4)

print("✅ Data Cleaning Complete! Saved as 'cleaned_transactions.csv'")
print(df.head())



✅ Data Cleaning Complete! Saved as 'cleaned_transactions.csv'
         Date                Transaction Details  Withdrawal (Dr)  \
0  2024-09-01         UPI/REF983457234 \nPayment              0.0   
1  2024-09-01                     ATM Withdrawal           2000.0   
2  2024-09-01           NEFT/INR/457832 \nSalary              0.0   
3  2024-09-01  IMPS/REF1029384 \nOnline Transfer           5000.0   
4  2024-09-02       UPI Payment: \nGrocery Store           2500.0   

   Deposit (Cr)  
0       50000.0  
1           0.0  
2       40000.0  
3           0.0  
4           0.0  


##Transaction Categorization

In [16]:
import pandas as pd


df = pd.read_csv("cleaned_transactions.csv")

def categorize_transaction(description):
    description = str(description).lower()

    if any(keyword in description for keyword in ["salary", "neft", "refund", "deposit"]):
        return "Income"
    elif any(keyword in description for keyword in ["upi", "grocery", "pos", "shopping", "bill"]):
        return "Expense"
    elif any(keyword in description for keyword in ["loan", "emi", "mortgage"]):
        return "Loan Payment"
    elif any(keyword in description for keyword in ["imps", "online transfer"]):
        return "Transfer"
    elif any(keyword in description for keyword in ["atm", "pos debit"]):
        return "Withdrawal"
    else:
        return "Uncategorized"


df["Category"] = df["Transaction Details"].apply(categorize_transaction)


df.to_csv("categorized_transactions.csv", index=False)
df.to_json("categorized_transactions.json", orient="records", indent=4)

print("✅ Categorization Complete! Saved as 'categorized_transactions.csv'")
print(df.head())


✅ Categorization Complete! Saved as 'categorized_transactions.csv'
         Date                Transaction Details  Withdrawal (Dr)  \
0  2024-09-01         UPI/REF983457234 \nPayment              0.0   
1  2024-09-01                     ATM Withdrawal           2000.0   
2  2024-09-01           NEFT/INR/457832 \nSalary              0.0   
3  2024-09-01  IMPS/REF1029384 \nOnline Transfer           5000.0   
4  2024-09-02       UPI Payment: \nGrocery Store           2500.0   

   Deposit (Cr)    Category  
0       50000.0     Expense  
1           0.0  Withdrawal  
2       40000.0      Income  
3           0.0    Transfer  
4           0.0     Expense  


##Build API Endpoints

In [10]:
pip install fastapi uvicorn pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.8 MB/s eta 0:00:00


In [12]:
pip install python-multipart

In [17]:
from fastapi import FastAPI, UploadFile, File, HTTPException
import pandas as pd
import shutil

app = FastAPI()


df = pd.read_csv("categorized_transactions.csv")


@app.get("/api/transactions")
def get_transactions():
    return df.to_dict(orient="records")

@app.get("/api/summary")
def get_summary():
    summary = {
        "Total Income": df[df["Category"] == "Income"]["Deposit (Cr)"].sum(),
        "Total Expenses": df[df["Category"] == "Expense"]["Withdrawal (Dr)"].sum(),
        "Total Loans Paid": df[df["Category"] == "Loan Payment"]["Withdrawal (Dr)"].sum(),
        "Net Balance": df["Deposit (Cr)"].sum() - df["Withdrawal (Dr)"].sum(),
    }
    return summary


@app.post("/api/upload")
async def upload_file(file: UploadFile = File(...)):
    if not file.filename.endswith(".csv"):
        raise HTTPException(status_code=400, detail="Only CSV files are allowed")

    file_location = f"uploaded_{file.filename}"
    with open(file_location, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)

    return {"message": f"File '{file.filename}' uploaded successfully"}


